<a href="https://colab.research.google.com/github/mar1po5-a/ezen/blob/main/konlpy%2Bgenai_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -q langchain-google-genai
!pip install -q langchain langchain-community chromadb
!pip install -q langchain-huggingface
!pip install -q konlpy

In [20]:
import os
from google.colab import userdata

In [21]:
# Gemini API 키 비밀관리 설정 후 진행
# userdata.get() -> 저장된 비밀 값을 안전하게 불러옴
try:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except userdata.SecretNotFoundError:
  print("ERROR: GOOGLE_API_KEY not found in Colab secrets. Pleas set it.")

In [22]:
# 기존 임베딩----------------------------------------------------
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# 벡터 DB
from langchain_community.vectorstores import Chroma
# 문서 표현
from langchain_core.documents import Document
# 텍스트 크기 분할
from langchain_text_splitters import KonlpyTextSplitter
# 프롬프트 생성
from langchain_core.prompts import ChatPromptTemplate
# 체인 내에서 데이터를 그대로 전달
from langchain_core.runnables import RunnablePassthrough
# LLM 출력을 문자열로 변환
from langchain_core.output_parsers import StrOutputParser

# 변경점----------------------------------------------------
import torch
from langchain_huggingface import HuggingFaceEmbeddings

In [23]:
# 샘플 데이터
documents_data = [
    "대한민국의 수도는 서울입니다. 서울은 역사와 현대가 공존하는 도시입니다.",
    "제주도는 아름다운 자연 경관으로 유명한 대한민국의 섬입니다. 한라산 국립공원이 대표적입니다.",
    "부산은 대한민국 제2의 도시이자 최대 항구 도시입니다. 해운대 해수욕장이 유명합니다.",
    "인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력 등을 컴퓨터 프로그램을 통해 구현하는 기술입니다.",
    "머신러닝은 인공지능의 한 분야로, 컴퓨터가 데이터를 통해 스스로 학습하고 예측하는 기술입니다."
    "［청년정책］　청년내일채움공제 [지원조건]　중소기업에 정규직으로 취업한 청년에 해당하는 사람 [수령금액] 1,500만 원을 수령하게 됩니다.",
    "지구 온난화는 전 지구적 기온 상승을 야기하며, 해수면 상승과 극심한 기상 현상의 원인이 됩니다. 탄소 배출량 감축이 시급한 과제입니다.",
    "블록체인 기술은 분산 원장 기술의 한 종류로, 데이터의 투명성과 보안성을 높이는 데 기여합니다. 암호화폐 외에도 다양한 분야에 활용될 수 있습니다.",
    "고대 로마 제국은 강력한 군사력과 법률 체계를 바탕으로 지중해 세계를 지배했으며, 서양 문명에 큰 영향을 미쳤습니다. 콜로세움과 판테온은 대표적인 건축물입니다.",
    """정책 제목: "2025 청년 디지털 커리어 점프업 패키지"
최근 디지털 전환 시대에 발맞춰 청년들의 성공적인 커리어 개발과 안정적인 사회 정착을 지원하기 위해 "2025 청년 디지털 커리어 점프업 패키지"가 새롭게 시행됩니다. 본 정책은 대한민국 국적을 가진 만 19세 이상 만 34세 이하의 미취업 또는 단기 근로 청년이라면 누구나 신청 가능하며, 특히 IT, AI, 빅데이터, 콘텐츠 제작 등 미래 유망 디지털 분야로의 진출을 희망하거나 해당 분야에서 역량을 강화하고자 하는 청년들을 우선적으로 지원합니다. 최종학력이나 전공에 관계없이 지원할 수 있으나, 관련 분야 교육 수료 경험이나 포트폴리오를 제출할 경우 가산점이 부여될 수 있습니다.
"2025 청년 디지털 커리어 점프업 패키지"의 주요 내용은 크게 세 가지로 구성됩니다. 첫째, '디지털 핵심역량 강화 교육 지원금'입니다. 선정된 청년에게는 최대 6개월간 매월 50만원의 교육 훈련비가 지급되어, 정부 인증 교육기관 또는 온라인 플랫폼에서 제공하는 고급 디지털 기술 교육 과정을 수강하는 데 사용할 수 있습니다. 이 지원금은 교육 과정의 종류나 기간에 따라 차등 지급될 수 있으며, 성실히 교육을 이수하고 수료증을 제출하는 청년에게는 추가적으로 50만원의 '역량 달성 축하금'이 일시 지급됩니다.
둘째, '미래설계 커리어 컨설팅 및 네트워킹 지원'입니다. 정책 참여 청년들은 전문 커리어 컨설턴트로부터 1:1 맞춤형 진로 상담, 이력서 및 자기소개서 첨삭, 모의 면접 등의 심층 컨설팅을 최소 3회 이상 제공받게 됩니다. 또한, 현업 전문가 및 동료 청년들과의 네트워킹 기회를 확대하기 위해 정기적인 온·오프라인 밋업(Meet-up) 및 워크숍 참여 비용의 80%를 지원하며, 연간 최대 20만원까지 지원받을 수 있습니다. 이를 통해 청년들은 실질적인 취업 정보를 얻고 인적 네트워크를 구축하는 데 도움을 받을 수 있습니다.
셋째, '디지털 분야 취·창업 초기 정착 지원금'입니다. 디지털 핵심역량 강화 교육을 성공적으로 수료하고, 해당 분야로 6개월 이내에 취업하거나 창업에 성공한 청년에게는 초기 정착을 돕기 위한 지원금이 지급됩니다. 취업자의 경우, 정규직으로 3개월 이상 근속 시 월 30만원씩 최대 3개월간, 총 90만원의 '취업 성공 장려금'을 받을 수 있습니다. 창업자의 경우에는 사업자 등록 후 초기 운영자금 명목으로 150만원을 일시금으로 지원하며, 이후 1년간 세무 및 법률 자문 서비스를 무료로 제공받을 수 있는 바우처도 함께 제공됩니다. 단, 이 지원금은 다른 정부 및 지자체의 유사 사업 지원금과 중복 수혜가 제한될 수 있으니 신청 시 유의해야 합니다.
본 정책의 신청 기간은 2025년 3월 1일부터 3월 31일까지이며, 공식 정책 안내 웹사이트를 통해 온라인으로만 접수 가능합니다. 제출 서류로는 신분증 사본, 최종학력 증명서, 구직활동 증명서(또는 단기 근로 계약서), 그리고 선택적으로 디지털 분야 관련 교육 수료증 및 포트폴리오 등이 요구됩니다. 선정 결과는 서류 심사 및 필요한 경우 비대면 인터뷰를 거쳐 5월 초에 개별 통보될 예정입니다. "2025 청년 디지털 커리어 점프업 패키지"를 통해 많은 청년들이 디지털 시대의 주역으로 성장할 수 있기를 기대합니다."""
]

In [24]:
# 샘플 데이터를 Document 객체로 변환
# page_context와 metadata를 가짐
docs = [Document(page_content=text, metadata={"source": f"doc_{i+1}"})
for i, text in enumerate(documents_data)]

print(docs)

[Document(metadata={'source': 'doc_1'}, page_content='대한민국의 수도는 서울입니다. 서울은 역사와 현대가 공존하는 도시입니다.'), Document(metadata={'source': 'doc_2'}, page_content='제주도는 아름다운 자연 경관으로 유명한 대한민국의 섬입니다. 한라산 국립공원이 대표적입니다.'), Document(metadata={'source': 'doc_3'}, page_content='부산은 대한민국 제2의 도시이자 최대 항구 도시입니다. 해운대 해수욕장이 유명합니다.'), Document(metadata={'source': 'doc_4'}, page_content='인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력 등을 컴퓨터 프로그램을 통해 구현하는 기술입니다.'), Document(metadata={'source': 'doc_5'}, page_content='머신러닝은 인공지능의 한 분야로, 컴퓨터가 데이터를 통해 스스로 학습하고 예측하는 기술입니다.［청년정책］\u3000청년내일채움공제 [지원조건]\u3000중소기업에 정규직으로 취업한 청년에 해당하는 사람 [수령금액] 1,500만 원을 수령하게 됩니다.'), Document(metadata={'source': 'doc_6'}, page_content='지구 온난화는 전 지구적 기온 상승을 야기하며, 해수면 상승과 극심한 기상 현상의 원인이 됩니다. 탄소 배출량 감축이 시급한 과제입니다.'), Document(metadata={'source': 'doc_7'}, page_content='블록체인 기술은 분산 원장 기술의 한 종류로, 데이터의 투명성과 보안성을 높이는 데 기여합니다. 암호화폐 외에도 다양한 분야에 활용될 수 있습니다.'), Document(metadata={'source': 'doc_8'}, page_content='고대 로마 제국은 강력한 군사력과 법률 체계를 바탕으로 지중해 세계를 지배했으며, 서양 문명에 큰

In [25]:
# 검색 정확도를 높이고, 토큰을 효율적으로 활용하기 위해 chunk 단위로 분할
text_splitter = KonlpyTextSplitter(
    chunk_size = 700, # 각 청크의 크기
    chunk_overlap = 70 # 청크 간 중복되는 글자 수
)
# split_documents -> 그냥 변수명
# .split_documents() -> method
split_documents = text_splitter.split_documents(docs)

print(f"원본 문서 개수: {len(docs)}")
print(f"분할된 청크 개수: {len(split_documents)}")
# print("\n분할된 첫 번째 청크 예시:")
print(split_documents[8])

원본 문서 개수: 9
분할된 청크 개수: 11
page_content='정책 제목: "2025 청년 디지털 커리어 점프 업 패키지" 최근 디지털 전환 시대에 발맞춰 청년들의 성공적인 커리어 개발과 안정적인 사회 정착을 지원하기 위해 "2025 청년 디지털 커리어 점프 업 패키지" 가 새롭게 시행됩니다.

본 정책은 대한민국 국적을 가진 만 19세 이상 만 34세 이하의 미 취업 또는 단기 근로 청년이라면 누구나 신청 가능하며, 특히 IT, AI, 빅 데이터, 콘텐츠 제작 등 미래 유망 디지털 분야로의 진출을 희망하거나 해당 분야에서 역량을 강화하고자 하는 청년들을 우선적으로 지원합니다.

최종 학력이나 전공에 관계없이 지원할 수 있으나, 관련 분야 교육 수료 경험이나 포트 폴리 오를 제출할 경우 가산점이 부여될 수 있습니다.

"2025 청년 디지털 커리어 점프 업 패키지" 의 주요 내용은 크게 세 가지로 구성됩니다.

첫째, ' 디지털 핵심 역량 강화 교육 지원금' 입니다.

선정된 청년에게는 최대 6개월 간 매월 50만원의 교육 훈련 비가 지급되어, 정부 인증 교육기관 또는 온라인 플랫폼에서 제공하는 고급 디지털 기술 교육 과정을 수강하는 데 사용할 수 있습니다.

이 지원 금은 교육 과정의 종류나 기간에 따라 차등 지급될 수 있으며, 성실히 교육을 이수하고 수료증을 제출하는 청년에게는 추가적으로 50만원의 ' 역 량 달성 축하금' 이 일시 지급됩니다.' metadata={'source': 'doc_9'}


In [26]:
# 변경점----------------------------------------------------
# 임베딩 모델 초기화
model_name_or_path = "nlpai-lab/KURE-v1"
# model_name_or_path = "kakaobank/kf-deberta-base"

# --- LangChain의 HuggingFaceEmbeddings 래퍼 사용 ---
# HuggingFaceEmbeddings 클래스가 내부적으로 모델 로드 및 임베딩 추출을 처리해줍니다.
try:
    # model_name에 Hugging Face Hub의 모델 ID 또는 로컬 경로를 지정합니다.
    # model_kwargs는 모델 로드 시 전달할 추가 인자입니다 (예: device 설정).
    # encode_kwargs는 텍스트를 인코딩(임베딩)할 때 전달할 인자입니다 (예: normalize_embeddings).
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name_or_path,
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}, # GPU 사용 가능하면 GPU, 아니면 CPU
        encode_kwargs={'normalize_embeddings': True} # 임베딩 정규화 (선택 사항, 성능에 영향 줄 수 있음)
    )
    # 간단한 테스트
    test_query_embedding = embeddings.embed_query("나이스")
    print(f"한국어 임베딩 모델 ('{model_name_or_path}')이 HuggingFaceEmbeddings를 통해 성공적으로 로드 및 테스트되었습니다.")
    print(f"테스트 임베딩 차원: {len(test_query_embedding)}")

except Exception as e:
    print(f"HuggingFaceEmbeddings 초기화 중 오류 발생: {e}")
    print("모델 이름이나 경로가 올바른지, 필요한 라이브러리(transformers, sentence-transformers 등)가 설치되었는지 확인해주세요.")
    raise

# llm 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    temperature=0.3 # 창의성
    # 시스템 프롬프트를 사용자 프롬프트 형태로 변환하여 전달
    # API 호환성과 model의 특성에 따라 유동적으로 선택하는 옵션
    # convert_system_message_to_human=True
)

print("Gemini 임베딩 모델 및 LLM이 성공적으로 초기화되었습니다.")

한국어 임베딩 모델 ('nlpai-lab/KURE-v1')이 HuggingFaceEmbeddings를 통해 성공적으로 로드 및 테스트되었습니다.
테스트 임베딩 차원: 1024
Gemini 임베딩 모델 및 LLM이 성공적으로 초기화되었습니다.


In [27]:
# 벡터 저장소 생성
# Chorma.from_documents()는 모든 작업을 한 번에 수행
# 1. 문서 청크를 임베딩 모델을 사용해 벡터로 변환
# 2. 벡터와 원본 텍스트, 메타 데이터를 ChromaDB에 저장
# persist_directory를 지정하면 해당 경로에 DB 파일이 저장됨. -> 재사용 가능
vectorstore_path = "./chroma_db_rag_store"
vectorstore = Chroma.from_documents(
    documents=split_documents, # 분할된 문서
    embedding=embeddings, # 임베딩 모델
    # persist : 지속하다
    persist_directory=vectorstore_path # DB 저장할 디렉토리
)

# 저장된 DB를 다시 로드할 때 사용 (재생성 x)
# vectorstore = Chroma(persist_directory=vectorstore_path, embedding_function=embeddings)

print(f"문서들이 ChromaDB에 성공적으로 임베딩 및 저장되었습니다. (저장 경로: {vectorstore_path})")
# 기존의 RDB : table / Vector DB : collection
print(f"ChromaDB에 저장된 총 벡터(청크) 수: {vectorstore._collection.count()}")

문서들이 ChromaDB에 성공적으로 임베딩 및 저장되었습니다. (저장 경로: ./chroma_db_rag_store)
ChromaDB에 저장된 총 벡터(청크) 수: 22


In [28]:
# 벡터 저장소에서 유사도 높은 문서를 검색하는 검색기 생성
# retriever : 되찾다, 회수하다
# k=3은 가장 유사한 문서 3개를 가져오도록 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 검색기 테스트
query_test = "대한민국의 수도는 어디인가요?"
retrieved_docs_test = retriever.invoke(query_test)
print(f"'{query_test}'에 대한 검색 결과 (상위 {len(retrieved_docs_test)}개)")
for i, doc in enumerate(retrieved_docs_test):
  # 메타데이터 중 source를 가져오되, 존재하지 않으면 기본 값을 가져옴
  print(f" {i+1}, {doc.page_content} (출처: {doc.metadata.get('source', 'N/A')})")

'대한민국의 수도는 어디인가요?'에 대한 검색 결과 (상위 3개)
 1, 대한민국의 수도는 서울입니다.

서울은 역사와 현대가 공존하는 도시입니다. (출처: doc_1)
 2, 대한민국의 수도는 서울입니다.

서울은 역사와 현대가 공존하는 도시입니다. (출처: doc_1)
 3, 부산은 대한민국 제 2의 도시 이자 최대 항구 도시입니다.

해운대 해수욕장이 유명합니다. (출처: doc_3)


In [29]:
# RAG 파이프라인 구성 : 정보 검색 - 문맥 정보 준비 - 프롬프트 구성
# - 답변 생성 및 출력 일련의 과정이 파이프라인

# 프롬프트 템플릿 정의
# {context} : 검색된 문서 내용, {question} : 사용자 질
prompt_template = """당신은 질문에 친절하고 상세하게 답변하는 AI 어시스턴트입니다.
주어진 문맥(context) 정보를 바탕으로 질문에 답변해주세요. 문맥에서 답을 찾을 수 없다면, "제공된 정보만으로는 답변하기 어렵습니다."라고 솔직하게 말해주세요.

문맥:
{context}

질문:
{question}

답변:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# RAG 체인 정의
# retriever로 문서를 검색하고
# 검색된 문서들의 page_content를 하나의 문자열로 합
# LCEL 사용 : |(파이프 기호)의 사용, 파이프라인 또는 체인을 더 쉽고 유연하게 구성하는 방식
rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
    # RunnablePassthrough.assian() : 체인 내에서 새로운 키를 생성하고 값을 할당
     # question에 초기 입력을 그대로 할당하기 위해 키:값으로 세팅
     "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser() # llm 출력을 일반 문자열로 변환
)

print("RAG 체인이 성공적으로 구성되었습니다.")


RAG 체인이 성공적으로 구성되었습니다.


In [30]:
# RAG 체인에 질문을 입력하여 답변 생성

question1 = "대한민국의 수도는 어디이고, 그 도시는 어떤 특징이 있나요?"
print(f"\n[질문 1]: {question1}")
answer1 = rag_chain.invoke(question1)
print(f"[답변 1]: {answer1}")

question2 = "블록체인 기술의 장점과 활용 분야에 대해 알려주세요."
print(f"\n[질문 2]: {question2}")
answer2 = rag_chain.invoke(question2)
print(f"[답변 2]: {answer2}")

question3 = "서울의 유명한 건축물은 무엇인가요?" # 샘플 문서에 없는 내용
print(f"\n[질문 3]: {question3}")
answer3 = rag_chain.invoke(question3)
print(f"[답변 3]: {answer3}")

question4 = "고대 로마 제국이 지중해를 지배할 수 있었던 배경과 대표적인 건축물은 무엇인가요?"
print(f"\n[질문 4]: {question4}")
answer4 = rag_chain.invoke(question4)
print(f"[답변 4]: {answer4}")

question5 = "청년내일채움공제 지원조건"
print(f"\n[질문 5]: {question5}")
answer5 = rag_chain.invoke(question5)
print(f"[답변 5]: {answer5}")


[질문 1]: 대한민국의 수도는 어디이고, 그 도시는 어떤 특징이 있나요?
[답변 1]: 제공된 정보에 따르면, 대한민국의 수도는 서울입니다.

서울은 역사와 현대가 공존하는 도시입니다.

[질문 2]: 블록체인 기술의 장점과 활용 분야에 대해 알려주세요.
[답변 2]: 제공된 문맥에 따르면, 블록체인 기술의 장점과 활용 분야는 다음과 같습니다.

*   **장점:** 데이터의 투명성과 보안성을 높이는 데 기여합니다.
*   **활용 분야:** 암호화폐 외에도 다양한 분야에 활용될 수 있습니다.

문맥에서는 블록체인 기술의 구체적인 활용 분야 목록을 제공하고 있지는 않습니다.

[질문 3]: 서울의 유명한 건축물은 무엇인가요?
[답변 3]: 제공된 정보만으로는 답변하기 어렵습니다.

[질문 4]: 고대 로마 제국이 지중해를 지배할 수 있었던 배경과 대표적인 건축물은 무엇인가요?
[답변 4]: 제공된 문맥에 따르면, 고대 로마 제국은 **강력한 군사력**과 **법률 체계**를 바탕으로 지중해 세계를 지배할 수 있었습니다.

대표적인 건축물로는 문맥에서 언급된 **콜로세움**과 **판테온**이 있습니다.

[질문 5]: 청년내일채움공제 지원조건
[답변 5]: 제공된 정보만으로는 "청년내일채움공제"의 지원조건에 대해 답변하기 어렵습니다.

제공된 문맥은 "2025 청년 디지털 커리어 점프 업 패키지"라는 다른 정책에 대한 정보만을 포함하고 있습니다.


In [31]:
questions_list = [
    # --- 새로운 청년 정책 관련 질문 (6번부터) ---
    "2025 청년 디지털 커리어 점프업 패키지에 지원하려면 어떤 나이 조건과 국적 조건이 필요한가요?", # 질문 6
    "디지털 핵심역량 강화 교육 지원금은 매월 얼마씩, 최대 몇 개월 동안 받을 수 있나요? 그리고 추가적인 혜택도 있나요?", # 질문 7
    "커리어 컨설팅 지원 내용과 네트워킹 지원 시 비용 지원 한도는 어떻게 되나요?", # 질문 8
    "디지털 분야 취업 성공 시 받게 되는 장려금의 총액과 지급 조건은 무엇인가요? 창업 지원금은 얼마인가요?", # 질문 9
    "2025 청년 디지털 커리어 점프업 패키지의 신청 기간과 필요한 주요 제출 서류는 무엇인가요?" # 질문 10
]

# 각 질문에 대해 RAG 체인을 실행하고 답변을 출력합니다.
for i, question_text in enumerate(questions_list):
    # 질문 번호는 실제 리스트 인덱스에 맞춰서 출력 (또는 원하는 시작 번호로 조정)
    print(f"\n[질문 {i + 1}]: {question_text}") # 여기서는 리스트 순서대로 1번부터 출력
    # 만약 6번부터 번호를 매기고 싶다면 print(f"\n[질문 {i + 6}]: {question_text}") 와 같이 조정
    answer = rag_chain.invoke(question_text)
    print(f"[답변 {i + 1}]: {answer}")


[질문 1]: 2025 청년 디지털 커리어 점프업 패키지에 지원하려면 어떤 나이 조건과 국적 조건이 필요한가요?
[답변 1]: 네, 친절하게 답변해 드리겠습니다.

제공된 문맥 정보에 따르면, "2025 청년 디지털 커리어 점프 업 패키지"에 지원하기 위한 나이 조건과 국적 조건은 다음과 같습니다.

*   **국적 조건:** 대한민국 국적을 가지고 있어야 합니다.
*   **나이 조건:** 만 19세 이상 만 34세 이하의 청년이어야 합니다.

따라서 대한민국 국적을 가진 만 19세 이상 만 34세 이하의 미취업 또는 단기 근로 청년이라면 누구나 신청 가능합니다.

[질문 2]: 디지털 핵심역량 강화 교육 지원금은 매월 얼마씩, 최대 몇 개월 동안 받을 수 있나요? 그리고 추가적인 혜택도 있나요?
[답변 2]: 제공된 문맥에 따르면, '디지털 핵심 역량 강화 교육 지원금'은 다음과 같이 받을 수 있습니다.

*   **매월 지급액:** 매월 50만원의 교육 훈련비가 지급됩니다.
*   **최대 지급 기간:** 최대 6개월 간 받을 수 있습니다.

추가적인 혜택도 있습니다. 교육을 성실히 이수하고 수료증을 제출하는 청년에게는 추가적으로 50만원의 '역량 달성 축하금'이 일시금으로 지급됩니다.

[질문 3]: 커리어 컨설팅 지원 내용과 네트워킹 지원 시 비용 지원 한도는 어떻게 되나요?
[답변 3]: 제공된 문맥에 따르면, '미래설계 커리어 컨설팅 및 네트워킹 지원' 내용은 다음과 같습니다.

1.  **커리어 컨설팅 지원 내용:**
    *   전문 커리어 컨설턴트로부터 1:1 맞춤형 진로 상담을 제공받습니다.
    *   이력서 및 자기소개서 첨삭 지원을 받을 수 있습니다.
    *   모의 면접을 통해 면접 준비를 할 수 있습니다.
    *   이러한 심층 컨설팅은 최소 3회 이상 제공됩니다.

2.  **네트워킹 지원 시 비용 지원 한도:**
    *   정기적인 온·오프라인 밋업(Meet-up) 및 워크숍 참여 비용의 80%를 지원합니다.
 

In [ ]:
# 셀 8 (또는 셀 10): RAG 체인 실행 (질문하고 답변 받기) - 문맥 확인 코드 추가

# 각 질문에 대해 RAG 체인을 실행하고 답변을 출력합니다.
for i, question_text in enumerate(questions_list):
    question_number = i + 1 # 실제 질문 번호 (1부터 시작)
    # 만약 6번부터 번호를 매기고 싶다면 question_number = i + 6

    print(f"\n\n--- [질문 {question_number}: {question_text}] ---")

    # 1. 해당 질문에 대한 문맥(검색된 문서) 확인
    # Retriever를 직접 호출하여 검색 결과를 가져옵니다.
    # rag_chain 내부의 retriever와 동일한 retriever 객체를 사용해야 합니다.
    # (이전 셀에서 'retriever'라는 이름으로 생성했다고 가정)
    try:
        retrieved_docs = retriever.invoke(question_text) # retriever 직접 호출
        print(f"\n[검색된 문맥 정보 (상위 {len(retrieved_docs)}개)]:")
        if not retrieved_docs:
            print("  -> 검색된 문서가 없습니다.")
        for j, doc in enumerate(retrieved_docs):
            print(f"  [문맥 {j+1}] 출처: {doc.metadata.get('source', 'N/A')}")
            print(f"  내용: {doc.page_content}\n")

        # 검색된 문서 내용을 하나의 문자열로 합치기 (RAG 체인 내부 로직과 유사하게)
        context_for_llm = "\n\n".join(d.page_content for d in retrieved_docs)
        # print(f"\n[LLM에 전달될 최종 문맥]:\n{context_for_llm}") # 너무 길면 주석 처리

    except Exception as e:
        print(f"  -> 문맥 검색 중 오류 발생: {e}")
        context_for_llm = "문맥 정보를 가져오는 데 실패했습니다." # 오류 시 기본 문맥

    # 2. RAG 체인 실행하여 답변 생성
    # 이제 rag_chain은 내부적으로 retriever를 다시 호출하겠지만,
    # 우리는 이미 어떤 문맥이 사용될지 확인했습니다.
    # 만약 retriever 호출을 한 번만 하고 싶다면, rag_chain 구성을 약간 수정해야 합니다.
    # (예: retriever 결과를 직접 rag_chain의 context 입력으로 전달)
    # 여기서는 단순 확인을 위해 기존 rag_chain을 그대로 사용합니다.
    print(f"\n[LLM 답변 생성 중...]")
    try:
        answer = rag_chain.invoke(question_text)
        print(f"[답변 {question_number}]: {answer}")
    except Exception as e:
        print(f"  -> LLM 답변 생성 중 오류 발생: {e}")
        print(f"[답변 {question_number}]: 답변을 생성하는 데 실패했습니다.")



--- [질문 1: 2025 청년 디지털 커리어 점프업 패키지에 지원하려면 어떤 나이 조건과 국적 조건이 필요한가요?] ---

[검색된 문맥 정보 (상위 3개)]:
  [문맥 1] 출처: doc_9
  내용: 정책 제목: "2025 청년 디지털 커리어 점프 업 패키지" 최근 디지털 전환 시대에 발맞춰 청년들의 성공적인 커리어 개발과 안정적인 사회 정착을 지원하기 위해 "2025 청년 디지털 커리어 점프 업 패키지" 가 새롭게 시행됩니다.

본 정책은 대한민국 국적을 가진 만 19세 이상 만 34세 이하의 미 취업 또는 단기 근로 청년이라면 누구나 신청 가능하며, 특히 IT, AI, 빅 데이터, 콘텐츠 제작 등 미래 유망 디지털 분야로의 진출을 희망하거나 해당 분야에서 역량을 강화하고자 하는 청년들을 우선적으로 지원합니다.

최종 학력이나 전공에 관계없이 지원할 수 있으나, 관련 분야 교육 수료 경험이나 포트 폴리 오를 제출할 경우 가산점이 부여될 수 있습니다.

"2025 청년 디지털 커리어 점프 업 패키지" 의 주요 내용은 크게 세 가지로 구성됩니다.

첫째, ' 디지털 핵심 역량 강화 교육 지원금' 입니다.

선정된 청년에게는 최대 6개월 간 매월 50만원의 교육 훈련 비가 지급되어, 정부 인증 교육기관 또는 온라인 플랫폼에서 제공하는 고급 디지털 기술 교육 과정을 수강하는 데 사용할 수 있습니다.

이 지원 금은 교육 과정의 종류나 기간에 따라 차등 지급될 수 있으며, 성실히 교육을 이수하고 수료증을 제출하는 청년에게는 추가적으로 50만원의 ' 역 량 달성 축하금' 이 일시 지급됩니다.

  [문맥 2] 출처: doc_9
  내용: 단, 이 지원금은 다른 정부 및 지자체의 유사 사업 지원금과 중복 수혜가 제한될 수 있으니 신청 시 유의해야 합니다.

본 정책의 신청 기간은 2025년 3월 1일부터 3월 31 일까지이며, 공식 정책 안내 웹사이트를 통해 온라인으로만 접수 가능합니다.

제출 서류로는 신분증 사본, 최종 학력 증명서, 구직활동 증명서( 